## Day 46 Lecture 2 Assignment

In this assignment, we will calculate a similarity matrix using responses to a survey about student life at a university. We will perform clustering on this dataset later on.

In [174]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [175]:
# answer goes here
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')
df.head(2)


,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,Q3-Interested in Volunteering,Q3-Interested in Others,Q4-Passionate about Animal welfare,Q4-Passionate about Arts/Culture/Heritage,Q4-Passionate about Children/Youth,Q4-Passionate about Community building,"Q4-Passionate about Diversity & Inclusion (e.g. special needs, migrant worker, interfaith and intercultural understanding)",Q4-Passionate about Environmental sustainability,Q4-Passionate about Families,Q4-Passionate about Health/Well-being (e.g mental health),Q4-Passionate about Seniors,Q4-Passionate about Poverty reduction,Q4-Passionate about Education,Q4-Passionate about None of the above,Q4-Passionate about Others,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,2


For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [190]:
# answer goes here

df_q5 = df.copy()
for col in df.columns:
  if 'Q5' not in col:
    df_q5.drop(col, axis= 1, inplace= True)

df_q5.head(10)



,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,1,0,0
3,1,1,0,1,1,1,1,0,1,0
4,0,1,1,0,1,0,0,0,1,0
5,0,1,0,0,1,0,0,0,0,0
6,0,1,1,0,0,0,0,1,0,0
7,1,0,0,0,0,0,0,1,0,0
8,1,1,0,0,1,1,0,0,1,0
9,1,1,1,0,0,0,0,1,1,0


We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

The pdist function is a simple way to accomplish this goal, and the squareform function will transform the output of the pdist function into a tabular format that can be convert to a dataframe more easily.

In [177]:
# answer goes here
dist = pd.DataFrame(squareform(pdist(df_q5)))

dist.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2918,2919,2920,2921,2922,2923,2924,2925,2926,2927,2928,2929,2930,2931,2932,2933,2934,2935,2936,2937,2938,2939,2940,2941,2942,2943,2944,2945,2946,2947,2948,2949,2950,2951,2952,2953,2954,2955,2956,2957
0,0.000000,0.000000,1.000000,2.449490,1.732051,1.000000,1.414214,1.732051,2.000000,2.000000,1.732051,1.732051,2.236068,2.000000,1.414214,1.732051,1.732051,2.000000,2.000000,1.732051,1.732051,1.414214,1.732051,0.000000,1.732051,1.414214,1.732051,1.414214,1.732051,1.000000,1.732051,1.732051,2.236068,1.000000,1.732051,1.000000,1.000000,1.732051,1.732051,0.000000,...,1.414214,1.414214,1.414214,1.000000,0.000000,2.000000,1.000000,1.414214,1.000000,0.000000,2.000000,0.000000,1.000000,1.414214,1.000000,1.414214,1.414214,1.000000,0.000000,1.000000,1.732051,1.000000,0.000000,2.000000,0.000000,1.000000,1.414214,1.000000,1.414214,1.414214,1.000000,1.732051,1.414214,1.732051,1.732051,1.414214,0.000000,0.000000,1.414214,1.414214
1,0.000000,0.000000,1.000000,2.449490,1.732051,1.000000,1.414214,1.732051,2.000000,2.000000,1.732051,1.732051,2.236068,2.000000,1.414214,1.732051,1.732051,2.000000,2.000000,1.732051,1.732051,1.414214,1.732051,0.000000,1.732051,1.414214,1.732051,1.414214,1.732051,1.000000,1.732051,1.732051,2.236068,1.000000,1.732051,1.000000,1.000000,1.732051,1.732051,0.000000,...,1.414214,1.414214,1.414214,1.000000,0.000000,2.000000,1.000000,1.414214,1.000000,0.000000,2.000000,0.000000,1.000000,1.414214,1.000000,1.414214,1.414214,1.000000,0.000000,1.000000,1.732051,1.000000,0.000000,2.000000,0.000000,1.000000,1.414214,1.000000,1.414214,1.414214,1.000000,1.732051,1.414214,1.732051,1.732051,1.414214,0.000000,0.000000,1.414214,1.414214
2,1.000000,1.000000,0.000000,2.645751,2.000000,1.414214,1.000000,1.414214,2.236068,1.732051,1.414214,1.414214,2.000000,2.236068,1.732051,1.414214,1.414214,1.732051,1.732051,2.000000,1.414214,1.732051,1.414214,1.000000,1.414214,1.732051,1.414214,1.000000,2.000000,0.000000,1.414214,1.414214,2.000000,1.414214,2.000000,0.000000,1.414214,1.414214,2.000000,1.000000,...,1.732051,1.000000,1.000000,0.000000,1.000000,1.732051,0.000000,1.000000,1.414214,1.000000,1.732051,1.000000,1.414214,1.732051,1.414214,1.000000,1.000000,0.000000,1.000000,0.000000,1.414214,1.414214,1.000000,1.732051,1.000000,1.414214,1.732051,0.000000,1.732051,1.000000,1.414214,2.000000,1.732051,2.000000,2.000000,1.732051,1.000000,1.000000,1.000000,1.000000
3,2.449490,2.449490,2.645751,0.000000,2.236068,2.236068,2.828427,2.645751,1.414214,2.449490,2.645751,2.236068,2.236068,2.000000,2.449490,2.645751,2.645751,2.449490,2.449490,2.645751,2.645751,2.449490,2.645751,2.449490,2.236068,2.449490,2.645751,2.828427,2.236068,2.645751,2.645751,2.645751,2.645751,2.236068,2.236068,2.645751,2.645751,2.645751,2.645751,2.449490,...,2.449490,2.449490,2.828427,2.645751,2.449490,2.000000,2.645751,2.449490,2.645751,2.449490,2.000000,2.449490,2.236068,2.449490,2.236068,2.828427,2.449490,2.645751,2.449490,2.645751,2.236068,2.236068,2.449490,2.449490,2.449490,2.645751,2.449490,2.645751,2.000000,2.449490,2.645751,2.236068,2.000000,2.236068,1.732051,2.449490,2.449490,2.449490,2.449490,2.828427
4,1.732051,1.732051,2.000000,2.236068,0.000000,1.414214,1.732051,2.449490,1.732051,1.732051,2.000000,2.000000,2.000000,2.236068,2.236068,2.449490,2.000000,2.236068,1.732051,2.000000,1.414214,1.732051,2.000000,1.732051,2.000000,1.732051,1.414214,1.732051,2.000000,2.000000,2.000000,1.414214,2.000000,2.000000,2.000000,2.000000,1.414214,2.000000,2.000000,1.732051,...,1.732051,1.732051,1.732051,2.000000,1.732051,1.732051,2.000000,2.236068,1.414214,1.732051,1.732051,1.732051,1.414214,1.732051,2.000000,2.236068,2.236068,2.000000,1.732051,2.000000,2.000000,2.000000,1.732051,2.236068,1.732051,1.414214,1.732051,2.000000,1.732051,2.236068,1.414214,1.414214,1.732051,2.000000,1.414214,1.732051,1.732051,1.732051,2.236068,1.732051


Print the answers to the following three questions:

1. How many pairs of students are there in our dataset? (Remember to ignore a student pairing with themselves, and avoid double-counting - '3-5' is the same as '5-3'.) 
2. Of these pairs, how many gave identical answers? 
3. How many pairs had no answers in common?

Tip: it may help to first convert the NxN similarity matrix into a table with three columns: response_id_1, response_id_2, and the similarity between them. The new table should have NxN rows and 3 columns.

In [ ]:
# # answer goes here
# n = 0
# ident = 0
# disimilar = 0
# for i in range(dist.shape[0]-1):
#   for j in range(i+1, dist.shape[1]):
#     n += 1
#     if dist.iloc[i,j] == 0:
#       ident += 1
#     if dist.iloc[i,j] > 3.16:
#       disimilar += 1


In [179]:
dist['id'] = df.index

In [180]:
distmelt = dist.melt(id_vars = 'id', value_vars = dist.columns[dist.columns != 'id'])

In [181]:
distmelt.columns = ['id1', 'id2', 'similarity']
nopair = distmelt[distmelt['id1'] != distmelt['id2']]
similar = nopair[nopair['similarity'] == 0]['similarity'].count()/2

print(((2958*(2958-1))/2)-2958)
print(similar)
print(nopair[nopair['similarity'] > 3.16]['similarity'].count()/2)
nopair.similarity.mean()

4370445.0
136732.0
291.0


1.6949290642453516

For each student, calculate their average similarity with all other students. Then print the following:

1. On average, which student (i.e. which response_id) had the most similar responses to the survey population as a whole? What was the average similarity for this student, and what were their answers?
2. Repeat part 1, but for the student with the least similar responses to the survey population as a whole.
3. What do we notice about these students? What broader inferences can we make about the student population as a whole?

In [212]:
# answer goes here
from scipy.spatial.distance import hamming
for x in df_q5.columns:
  print(df_q5[x].value_counts())
popscore = df_q5.mode()

results = []

for index, row in df_q5.iterrows():
  hamming(row, popscore)
  results.append([index, hamming(row, popscore)])

0    2051
1     907
Name: Q5-Stressed about Adjustment issues, dtype: int64
1    2728
0     230
Name: Q5-Stressed about Academic issues, dtype: int64
0    1816
1    1142
Name: Q5-Stressed about Financial issues, dtype: int64
0    2522
1     436
Name: Q5-Stressed about Family issues, dtype: int64
0    2160
1     798
Name: Q5-Stressed about Friendships, dtype: int64
0    2468
1     490
Name: Q5-Stressed about Romantic relationships, dtype: int64
0    2511
1     447
Name: Q5-Stressed about  Health related issues, dtype: int64
0    1612
1    1346
Name: Q5-Stressed about Career related issues, dtype: int64
0    2147
1     811
Name: Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc., dtype: int64
0    2917
1      41
Name: Q5-Stressed about Others, dtype: int64


In [214]:
pd.DataFrame(data = results, columns = ['students', 'similarity to average'])

,students,similarity to average
0,0,0.0
1,1,0.0
2,2,0.1
3,3,0.6
4,4,0.3
...,...,...
2953,2953,0.2
2954,2954,0.0
2955,2955,0.0
2956,2956,0.2
